# Cutting Stock Column Generation

We will implement the master and pricing problems studied during the course.
The master problem selects the board configurations which are generated by the sub-problem.

#### Master problem:

\begin{align*}
\min & \sum_c x_c & \\
\text{subject to:} && \\
& \sum_c n_{cs} x_c \geq d_s & \forall s  \quad (\Pi^*_s) \\
& x_c \in \mathbb{N} & \forall c
\end{align*}

#### Pricing problem:

\begin{align*}
\min \quad & 1 - \sum_s \Pi^*_s n_s & \\
\text{subject to:} && \\
& \sum_s l_s n_s \leq L & \\
& n_s \in \mathbb{N} & \forall s
\end{align*}

The core procedure of the column generation can be described as follow:
1. Create the master problem with some initial columns to have a feasible problem;
2. Solve the linear relaxation of the master problem;
3. Update the sub-problem with the dual variables and solve it;
4. If the objective of the pricing problem is negative, add the corresponding configuration as a new column to the master problem;
5. If any column has been added to the master problem, go to 2, otherwise stop.

Finally, we will create a master problem with all the columns that has been generated by the column generation and we will solve the integer problem. We should normally obtain a good solution of the Cutting Stock problem. The objective value of the column generation is a lower bound that can be used to estimate the quality of the solution found.

In [20]:
L = 110
size = [20, 45, 50, 55, 75]
demand = [48, 35, 24, 10, 8]
nbShelves = len(size)
Shelves = range(nbShelves)

### Exercices

#### 1. Implement the pricing, and then the master porblems with the column generation logic.

In [21]:
from docplex.mp.model import Model

def subproblem(duals):
    pricing = Model(name='cutting_stock_subproblem')

    n = pricing.integer_var_list(nbShelves, lb=0, name="c")
    pricing.add_constraint(sum(size[i] * n[i] for i in Shelves) <= L, 'size')
    pricing.minimize(1 - sum(duals[i] * n[i] for i in Shelves))
    pricing.solve()
    return (pricing.objective_value, [int(n[i].solution_value) for i in Shelves])

Test the subproblem function:

In [22]:
subproblem(duals=[1] * nbShelves)

(-4, [5, 0, 0, 0, 0])

#### 2. Implement the master and solve it with some initial columns (steps 1-2 of the procedure)

In [23]:
#initial configuration
configs = [[L // s if r == s else 0 for r in size] for s in size]

master = Model(name='cutting_stock_master')
x = master.continuous_var_list(configs, lb=0, name="x")

demand_ctr = [0 for i in Shelves]
for i in Shelves:
    demand_ctr[i] = master.add_constraint(sum(configs[j][i] * x[j] for j in range(len(configs))) >= demand[i],'demand_%d' % i)

master.minimize(sum(x))
msol = master.solve()
print(msol)

solution for: cutting_stock_master
objective: 52.1
x_[5, 0, 0, 0, 0]=9.600
x_[0, 2, 0, 0, 0]=17.500
x_[0, 0, 2, 0, 0]=12.000
x_[0, 0, 0, 2, 0]=5.000
x_[0, 0, 0, 0, 1]=8.000



#### 3. Implement the column generation (step 1 - 5)
You should use the previous question.

In [24]:
# 1. Build the master problem

#initial configuration
configs = [[L // s if r == s else 0 for r in size] for s in size]

master = Model(name='cutting_stock_master')
x = master.continuous_var_list(configs, lb=0, name="x")

demand_ctr = [0 for i in Shelves]
for i in Shelves:
    demand_ctr[i] = master.add_constraint(sum(configs[j][i] * x[j] for j in range(len(configs))) >= demand[i],'demand_%d' % i)

master.minimize(sum(x))

iteration = 0
print('------------------------------------------------------------------')

while True:
    # 2. Solve the linear relaxation of the master problem
    # TODO: solve the linear relaxation of the master problem with the new columns
    
    master.solve()

    print('Iter={:5d}, Master obj={:.3f}, Columns: total={:6d}'.format(
        iteration, master.objective_value, len(configs)))
    print('------------------------------------------------------------------')
    
    # 3. Solve the sub-problem with the dual variables
    # TODO: solve the sub-problem
    
    duals = [d.dual_value for d in demand_ctr]
    (obj, c) = subproblem(duals)
    
    # 4. If the objective is negative, add the the new column to the master problem
    # TODO: add the column of necessary
    if (obj < 0):
        configs.append(c)
        v = master.continuous_var(lb=0, name='x_%d' % len(configs));
        x.append(v)
        for i in range(nbShelves):
            demand_ctr[i].left_expr.add_term(v, c[i])
        master.objective_expr.add_term(v, 1)
        print('Added config %d: size %2d, configuration %s' % (len(configs), sum(size[i]*c[i] for i in Shelves), c))
        n_columns = len(configs)
    else:
        break
    iteration += 1
    

# Print the solution
for i in range(len(configs)):
    s = x[i].solution_value
    if s > 0:
        print('size %3d, orders %s: %.2f' % (sum(configs[i][j]*c[j] for j in Shelves), c, s))

------------------------------------------------------------------
Iter=    0, Master obj=52.100, Columns: total=     5
------------------------------------------------------------------
Added config 6: size 95, configuration [1, 0, 0, 0, 1]
Iter=    1, Master obj=50.500, Columns: total=     6
------------------------------------------------------------------
Added config 7: size 110, configuration [1, 2, 0, 0, 0]
Iter=    2, Master obj=47.000, Columns: total=     7
------------------------------------------------------------------
Added config 8: size 110, configuration [3, 0, 1, 0, 0]
Iter=    3, Master obj=46.250, Columns: total=     8
------------------------------------------------------------------
size   0, orders [1, 0, 0, 0, 1]: 8.25
size   0, orders [1, 0, 0, 0, 1]: 5.00
size   2, orders [1, 0, 0, 0, 1]: 8.00
size   1, orders [1, 0, 0, 0, 1]: 17.50
size   3, orders [1, 0, 0, 0, 1]: 7.50


### Solve the Cutting Stock Problem
We use the column generated, to solve the integer problem. So, we must:
1. Redefine a integer problem with all the columns at the beginning;
2. Solve this problem

In [25]:
mdl = Model(name='cutting_stock')

x = mdl.integer_var_list(len(configs), lb=0, name="x")
for i in Shelves:
    mdl.add_constraint(sum(configs[j][i] * x[j] for j in range(len(configs))) >= demand[i],'demand_%d' % i)
mdl.minimize(sum(x))
msol = mdl.solve()
print('obj={}'.format(mdl.objective_value))
for i in range(len(configs)):
    s = x[i].solution_value
    if s > 0:
        print('size %3d, orders %s: %.2f' % (sum(configs[i][j]*c[j] for j in Shelves), c, s))

obj=47
size   0, orders [1, 0, 0, 0, 1]: 8.00
size   0, orders [1, 0, 0, 0, 1]: 5.00
size   2, orders [1, 0, 0, 0, 1]: 8.00
size   1, orders [1, 0, 0, 0, 1]: 18.00
size   3, orders [1, 0, 0, 0, 1]: 8.00
